# Library Import & Connection to DB

In [76]:
import psycopg2
from psycopg2 import Error
import pandas as pd
import pandas.io.sql as sqlio

from bokeh.plotting import figure, show
from bokeh.plotting import figure, output_file, show
from bokeh.palettes import Category20c, inferno
from bokeh.transform import cumsum
from bokeh.io import output_notebook

import numpy as np
import itertools
from datetime import datetime
from math import pi


# Postgres username, password, and database name
POSTGRES_ADDRESS = '172.16.169.33' 
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres' 
POSTGRES_PASSWORD = '1234' 
POSTGRES_DBNAME = 'postgres' 

# Create the connection
connection = psycopg2.connect(user=POSTGRES_USERNAME,
                              password=POSTGRES_PASSWORD,
                              host=POSTGRES_ADDRESS,
                              port=POSTGRES_PORT,
                              database=POSTGRES_DBNAME)

## Tuen-Ma Line

In [132]:
sql_1 = '''Select base_data.curr_time, up_data.ttnt from up_data, base_data 
                where up_data.base_id = base_data.base_id 
                    and base_data.curr_time >= '2022-04-22 17:15:00'
                    and base_data.curr_time < '2022-04-22 17:45:00'
                    and base_data.station = 'TML-TKW'
                    and up_data.seq = '1'
                order by base_data.curr_time asc;'''

df_1 = sqlio.read_sql_query(sql_1, connection)


# df_1a: Expected Waiting time pattern
# Add back missing departures by linear interpolation
df_1a = df_1.copy()
temp_1 = df_1a.loc[(df_1a['ttnt'].shift(-1) - df_1a['ttnt'] > 1) & (df_1a['ttnt'] != 0)].reset_index(drop=True)
temp_2 = df_1a.loc[(df_1a['ttnt'] - df_1a['ttnt'].shift(1) > 1) & (df_1a['ttnt'].shift(1) != 0)].reset_index(drop=True)
temp_1['curr_time'] = temp_1['curr_time'] + (temp_2['curr_time'] - temp_1['curr_time'])/2
temp_1['ttnt'] = 0

df_1a = df_1a.append(temp_1, ignore_index=True).sort_values(by=['curr_time']).reset_index(drop=True)
df_1a['curr_time'] = pd.to_datetime(df_1a['curr_time']).apply(lambda x: x.replace(microsecond=0))


# df_1b - Count number of TML train arrivals
# Drop extra ttnt = 0
df_1b = df_1a.copy()
df_1b = df_1b.drop(df_1b[(df_1b.diff().curr_time.dt.seconds < 120) & (df_1b['ttnt'] == 0) & (df_1b['ttnt'].shift(1) == 0)].index)
df_1b = df_1b[df_1b['ttnt'] == 0]
df_1b = df_1b.reset_index(drop=True)


**1. Display real waiting time patterns**

In [133]:
# df_tml_1 - Display TML waiting time patterns
df_tml_1 = df_1a.copy()
display(df_tml_1)

p_tml_1 = figure(width=800, height=400, x_axis_type="datetime")
p_tml_1.line(df_tml_1['curr_time'], df_tml_1['ttnt'])
show(p_tml_1)


,curr_time,ttnt
0,2022-04-22 17:15:07,3
1,2022-04-22 17:15:26,2
2,2022-04-22 17:16:03,2
3,2022-04-22 17:16:46,1
4,2022-04-22 17:17:04,1
5,2022-04-22 17:18:03,0
6,2022-04-22 17:18:32,3
7,2022-04-22 17:20:22,2
8,2022-04-22 17:21:59,1
9,2022-04-22 17:22:55,0


**2. Count number of TML arrivals**

In [134]:
# df_tml_2a - Count number of Light Rail arrivals (Multiple Bar Chart)
# Group train arrivals by time interval
df_tml_2a = df_1b.copy()
df_tml_2a = df_tml_2a.groupby([pd.Grouper(key='curr_time', freq='15Min')]).agg({'ttnt': 'count'}).reset_index()
display(df_tml_2a)

p_tml_2a = figure(width=600, height=600, x_axis_type="datetime")
p_tml_2a.vbar(source=df_tml_2a, x='curr_time', top='ttnt', width=24*60*15*10)
show(p_tml_2a)

,curr_time,ttnt
0,2022-04-22 17:15:00,2
1,2022-04-22 17:30:00,5


## Light-Rail

In [98]:
sql_2 = '''Select lr_base_data.sys_time, lr_route_list.route_no, lr_route_list.time_en, lr_route_list.train_length from lr_base_data, lr_route_list
                where lr_base_data.lr_data_id = lr_route_list.lr_data_id
                    and lr_base_data.sys_time >= '2022-04-22 14:15:00'
                    and lr_base_data.sys_time < '2022-04-22 14:45:00'
                    and lr_base_data.station_id = '435'
                    and lr_route_list.platform_id = '1'
                order by lr_base_data.sys_time asc;'''

df_2 = sqlio.read_sql_query(sql_2, connection)


# df_2a: Expected Waiting time pattern
# Change data fields to correct formating and type
df_2a = df_2.copy()
df_2a.loc[df_2a['time_en'] == 'Arriving', 'time_en'] = '1 min'
df_2a.loc[df_2a['time_en'] == 'Departing', 'time_en'] = '0 min'
df_2a.loc[df_2a['time_en'] == '-', 'time_en'] = '0 min'
df_2a['time_en'] = df_2a['time_en'].str.split(' ', 0).str[0]
df_2a = df_2a.astype({'sys_time': 'datetime64[ns]', 'route_no': 'int', 'time_en': 'int', 'train_length': 'int'})

# Reorder the data according to route and time
df_2a.sort_values(by=['sys_time', 'route_no', 'time_en'], inplace=True)
df_2a = df_2a.reset_index(drop=True)
df_2a = df_2a.groupby(['sys_time', 'route_no'], as_index=False).first()
df_2a.sort_values(by=['route_no', 'sys_time'], inplace=True)
df_2a = df_2a.reset_index(drop=True)

# Add back missing departures by linear interpolation
temp_1 = df_2a.loc[(df_2a['time_en'].shift(-1) - df_2a['time_en'] > 1) & (df_2a['time_en'] != 0) & (df_2a['route_no'].shift(-1) == df_2a['route_no'])].reset_index(drop=True)
temp_2 = df_2a.loc[(df_2a['time_en'] - df_2a['time_en'].shift(1) > 1) & (df_2a['time_en'].shift(1) != 0) & (df_2a['route_no'].shift(1) == df_2a['route_no'])].reset_index(drop=True)
temp_1['sys_time'] = temp_1['sys_time'] + (temp_2['sys_time'] - temp_1['sys_time'])/2
temp_1['time_en'] = 0

df_2a = df_2a.append(temp_1, ignore_index=True).sort_values(by=['route_no', 'sys_time']).reset_index(drop=True)
df_2a['sys_time'] = pd.to_datetime(df_2a['sys_time']).apply(lambda x: x.replace(microsecond=0))

display(df_2a)


# df_2b: Real Waiting time pattern
df_2b = df_2a.copy()

# Change to real waiting time
for i in df_2b.index:
    temp_3 = df_2b.iloc[i:]
    temp_3 = temp_3.loc[(temp_3['route_no'] == df_2b.at[i, 'route_no']) & (temp_3['time_en'] == 0)].head(1)
    
    if not temp_3.empty:
        time_diff = (temp_3['sys_time'] - df_2b.at[i, 'sys_time']).values[0].astype('timedelta64[s]').astype(int)
        time_diff = round(time_diff / 60)
        df_2b.at[i, 'time_en'] = time_diff

display(df_2b)
    

# df_2c: Arrival time pattern
# Drop extra time_en = 0
df_2c = df_2a.copy()
df_2c = df_2c.drop(df_2c[(df_2c.diff().sys_time.dt.seconds < 120) & (df_2c['time_en'] == 0) & (df_2c['time_en'].shift(1) == 0) & (df_2c['route_no'].shift(1) == df_2c['route_no'])].index)
df_2c = df_2c[df_2c['time_en'] == 0]
df_2c = df_2c.reset_index(drop=True)

display(df_2c)


,sys_time,route_no,time_en,train_length
0,2022-04-22 14:16:16,705,1,2
1,2022-04-22 14:16:50,705,0,2
2,2022-04-22 14:17:31,705,0,2
3,2022-04-22 14:18:05,705,6,2
4,2022-04-22 14:19:46,705,5,2
...,...,...,...,...
91,2022-04-22 14:42:28,751,2,1
92,2022-04-22 14:43:02,751,2,1
93,2022-04-22 14:43:29,751,1,1
94,2022-04-22 14:44:03,751,0,1


,sys_time,route_no,time_en,train_length
0,2022-04-22 14:16:16,705,1,2
1,2022-04-22 14:16:50,705,0,2
2,2022-04-22 14:17:31,705,0,2
3,2022-04-22 14:18:05,705,6,2
4,2022-04-22 14:19:46,705,4,2
...,...,...,...,...
91,2022-04-22 14:42:28,751,2,1
92,2022-04-22 14:43:02,751,1,1
93,2022-04-22 14:43:29,751,1,1
94,2022-04-22 14:44:03,751,0,1


,sys_time,route_no,time_en,train_length
0,2022-04-22 14:16:50,705,0,2
1,2022-04-22 14:23:54,705,0,2
2,2022-04-22 14:32:13,705,0,2
3,2022-04-22 14:38:29,705,0,2
4,2022-04-22 14:23:20,751,0,2
5,2022-04-22 14:33:55,751,0,2
6,2022-04-22 14:44:03,751,0,1


**1. Display real waiting time patterns**

In [99]:
# df_lr_1 - Display Light Rail real waiting time patterns (Line Chart)
df_lr_1 = df_2b.copy()

# Plot the graph
numLines = df_lr_1['route_no'].nunique()
colors = itertools.cycle(inferno(numLines))

p_lr_1 = figure(width=600, height=600, x_axis_type="datetime")
for name, group in df_lr_1.groupby('route_no'):
    p_lr_1.line(x=group.sys_time, y=group.time_en, legend_label=str(name), color=next(colors))

show(p_lr_1)


**2. Count number of Light Rail arrivals**

In [123]:
# df_lr_2a - Count number of Light Rail arrivals (Multiple Bar Chart)
# Group train arrivals by time interval
df_lr_2a = df_2c.copy()
df_lr_2a = df_lr_2a.groupby([pd.Grouper(key='sys_time', freq='15Min'), 'route_no']).agg({'time_en': 'count'}).reset_index()
display(df_lr_2a)

# Plot the graph
p_lr_2a = figure(width=600, height=600, x_axis_type="datetime")
i = -1
for name, group in df_lr_2a.groupby('route_no'):
    group['sys_time'] = group['sys_time'] + pd.to_timedelta(i, unit='m')
    p_lr_2a.vbar(source=group, x='sys_time', top='time_en', width=24*60*15*5, color=next(colors))
    i = i + 2
    
show(p_lr_2a)


# df_lr_2b - Count number of Light Rail arrivals (Pie Chart)
# Group train arrivals by time interval
df_lr_2b = df_2c.copy()
df_lr_2b = df_lr_2b.groupby([pd.Grouper(key='sys_time', freq='60Min'), 'route_no']).agg({'time_en': 'count'}).reset_index()
display(df_lr_2b)

# Plot the graph
chart_color = ['#44e5e2', '#e29e44']
df_lr_2b['angle'] = df_lr_2b['time_en'] / df_lr_2b['time_en'].sum() * 2 * pi
df_lr_2b['color'] = chart_color if len(df_lr_2b) <= 2 else Category20c[len(df_lr_2b)]

p_lr_2b = figure(width=600, height=600)
p_lr_2b.wedge(x=0, y=1, radius=0.4, start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
              line_color="white", fill_color='color', legend_field='route_no', source=df_lr_2b)
    
show(p_lr_2b)

,sys_time,route_no,time_en
0,2022-04-22 14:15:00,705,2
1,2022-04-22 14:15:00,751,1
2,2022-04-22 14:30:00,705,2
3,2022-04-22 14:30:00,751,2


C:\Users\Public\Anaconda3\envs\fyp\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,sys_time,route_no,time_en
0,2022-04-22 14:00:00,705,4
1,2022-04-22 14:00:00,751,3


**3. Display combined waiting time/frequency of light rail to a station**

In [126]:
# df_lr_3a - True waiting time of Light rail to a station (Line chart)
# Pick the earliest arrival at different periods
df_lr_3a = df_2b.copy()
df_lr_3a.sort_values(by=['sys_time', 'time_en'], inplace=True)
df_lr_3a = df_lr_3a.reset_index(drop=True)
df_lr_3a = df_lr_3a.groupby(['sys_time'], as_index=False).first()
display(df_lr_3a)

# Plot the graph
p_lr_3a = figure(width=800, height=400, x_axis_type="datetime")
p_lr_3a.line(df_lr_3a['sys_time'], df_lr_3a['time_en'])
show(p_lr_3a)


# df_lr_3b

,sys_time,route_no,time_en,train_length
0,2022-04-22 14:16:16,705,1,2
1,2022-04-22 14:16:50,705,0,2
2,2022-04-22 14:17:31,705,0,2
3,2022-04-22 14:18:05,751,5,2
4,2022-04-22 14:19:46,705,4,2
5,2022-04-22 14:20:18,751,3,2
6,2022-04-22 14:20:59,751,2,2
7,2022-04-22 14:21:32,705,2,2
8,2022-04-22 14:22:07,751,1,2
9,2022-04-22 14:22:32,705,1,2
